In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras.models import load_model

In [5]:
classes = None
with open('./models/coco.names', 'r') as f:
    classes = [line.strip() for line in f.readlines()]



# read pre-trained model and config file
net = cv2.dnn.readNet('./models/yolov3-tiny.weights', './models/yolov3-tiny.cfg')

# print(net.getLayerNames())
# create input blob 
# set input blob for the network


In [7]:
model = load_model('./models/model-005.model')

face_clsfr=cv2.CascadeClassifier('./models/haarcascade_frontalface_default.xml')



In [8]:
# source.read()
source=cv2.VideoCapture(0)

labels_dict={1:'MASK',0:'NO MASK'}
color_dict={1:(0,255,0),0:(0,0,255)}

In [9]:
# source.read()

In [10]:
threshold_for_green = 35
threshold_for_red = 10

red_time = 0
green_time = 0

door = 0

In [11]:
import time
number_of_frames = 0

while True:
    ###############################
    # HUMAN DETECTION 
    ################################
    
    #Start for fps counter calculation
    start_time = time.time()
    
    #Reading in the input image from camera
    ret,image=source.read()
    if(ret==False):
        break
    Width = image.shape[1]
    Height = image.shape[0]
    
    #Setting the input for our Human detection YOLO model
    net.setInput(cv2.dnn.blobFromImage(image, 0.00392, (416,416), (0,0,0), True, crop=False))

    
    # run inference through the network
    # and gather predictions from output layers
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []

    #create bounding box 
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.1:
                center_x = int(detection[0] * Width)
                center_y = int(detection[1] * Height)
                w = int(detection[2] * Width)
                h = int(detection[3] * Height)
                x = center_x - w / 2
                y = center_y - h / 2
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])

    
    #Thresholding the bounding box based on confidence scores
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.1, 0.1)
    
    print("{} these are the boxes".format(boxes))
    
    # If more than one humans detected
    if len(boxes):
        #Choose Human with highest probability
        x,y,w,h = boxes[indices[0][0]]
        
        # Crop Out the person from that image
        img = image[max(int(y),0):max(int(y),0)+h,max(int(x),0):max(int(x),0)+w]
        
        ################################
        # Face Detection
        ################################
        img = cv2.resize(img, (0,0), fx=image.shape[1]/img.shape[1], fy=image.shape[0]/img.shape[0]) 
        gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        faces=face_clsfr.detectMultiScale(gray,1.3,5)  
        
        #If face Detected
        if len(faces):
            x,y,w,h = faces[0]
            # Extract the face from bounding box and send to Mask Model for prediction
            face_img=gray[y:y+w,x:x+w]
            
            ###################################################
            # Mask Detection
            ###################################################
            resized=cv2.resize(face_img,(100,100))
            normalized=resized/255.0
            reshaped=np.reshape(normalized,(1,100,100,1))
            
            # Output of probabilities from our Mask Detection CNN model
            result=model.predict(reshaped)
            
            # Chose the label based on the higher probability
            label=np.argmax(result,axis=1)[0]
            
            # Creating System Code for door opening
            # Based on a continuous positive response(threshold_for_green) until a 
            # continuous Negative response(threshold_for_red) from system
            if(label == 0):
                red_time += 1
                if (red_time > threshold_for_red):
                    green_time = 0
                    red_time = 0
                    door = 0
            if(label == 1):
                green_time += 1
                if (green_time > threshold_for_green):
                    ref_time = 0
                    green_time = 0
                    door = 1

                    success = 100

                    ret,img=source.read()
                    cv2.rectangle(img, (0, 0), (img.shape[1], img.shape[0]), color_dict[door], 10)  
                    cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
                    cv2.imshow('LIVE',img)
                    cv2.waitKey(0)

                    cv2.destroyAllWindows()
                    break
            
            # Putting a rectangle on the detected face and the detected output
            cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
            cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
            cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)


        # A bounding rectangle for entire image 
        # whose color changes based on the variable door
        cv2.rectangle(img, (0, 0), (img.shape[1], img.shape[0]), color_dict[door], 10)

        #FPS counter calculation
        end_time = time.time()
        cv2.putText(img,"FPS: " + str(int(1/(end_time - start_time))),(20,50),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,200,0),2)
        
        cv2.imshow('LIVE',img)
        
        key=cv2.waitKey(1)
        
        # Break on press of the Escape Key
        if(key==27):
            break
            
    # If no human detected we just print the rad image from the camera
    else:
        ret,image=source.read()
        #FPS counter calculation
        end_time = time.time()
        cv2.putText(image,"FPS: " + str(int(1/(end_time - start_time))),(20,50),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,200,0),2)

        cv2.imshow('LIVE',image)
        
        key=cv2.waitKey(1)

        if(key==27):
            break
    
cv2.destroyAllWindows()
# source.release()

print(door)

[] these are the boxes
[] these are the boxes
[] these are the boxes
[] these are the boxes
[] these are the boxes
[] these are the boxes
[] these are the boxes
[] these are the boxes
[] these are the boxes
[] these are the boxes
[] these are the boxes
[] these are the boxes
[[127.5, 147.5, 475, 337]] these are the boxes
[] these are the boxes
[] these are the boxes
[] these are the boxes
[] these are the boxes
[] these are the boxes
[] these are the boxes
[] these are the boxes
[] these are the boxes
[] these are the boxes
[] these are the boxes
[] these are the boxes
[] these are the boxes
[] these are the boxes
[] these are the boxes
[] these are the boxes
[] these are the boxes
[] these are the boxes
[] these are the boxes
[] these are the boxes
[] these are the boxes
[] these are the boxes
[] these are the boxes
[] these are the boxes
[] these are the boxes
[[172.5, 225.5, 289, 259]] these are the boxes
[[181.0, 221.5, 276, 255]] these are the boxes
[] these are the boxes
[] these

[[91.0, 225.0, 462, 254]] these are the boxes
[[172.5, 219.0, 273, 270]] these are the boxes
[[171.0, 220.0, 276, 268]] these are the boxes
[[172.5, 222.5, 273, 265]] these are the boxes
[[174.5, 222.0, 269, 266]] these are the boxes
[[173.5, 221.0, 269, 268]] these are the boxes
[[174.0, 219.5, 272, 271]] these are the boxes
[[84.0, 227.0, 474, 250]] these are the boxes
[[176.0, 221.5, 266, 265]] these are the boxes
[[172.5, 216.5, 271, 275]] these are the boxes
[[92.5, 221.0, 457, 260]] these are the boxes
[[175.0, 220.5, 268, 271]] these are the boxes
[[159.0, 228.5, 250, 251]] these are the boxes
[[91.0, 223.5, 460, 259]] these are the boxes
[[173.0, 218.0, 272, 274]] these are the boxes
[[172.0, 219.5, 272, 269]] these are the boxes
[[87.5, 227.0, 467, 252]] these are the boxes
[[171.5, 218.0, 275, 274]] these are the boxes
[[170.5, 216.0, 275, 276]] these are the boxes
[[94.5, 228.0, 455, 248]] these are the boxes
[[91.0, 224.5, 460, 255]] these are the boxes
[[89.5, 226.5, 463, 